In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os
import numpy as np

In [10]:
# Cell 2: Load the datasets
base_path = "../DatabaseHelpers/resources/"
n_rows = 100_000

employees = pd.read_csv(base_path + 'employees.csv', names=['EmployeeId','FirstName','LastName','Gender','EmploymentDate','TerminationDate','Salary','RoleId','UserId']
                        # , nrows=n_rows
                        )
roles = pd.read_csv(base_path + 'employee_roles.csv', names=['RoleId','RoleName','Description','Level','UserId']
                    # , nrows=n_rows
                    )

employees.drop(columns=['EmployeeId','FirstName','LastName','Gender', 'UserId'], inplace=True)
roles.drop(columns=['Description','Level','UserId'], inplace=True)

In [11]:
# Cell 3: Preprocess the data
# Merge datasets on RoleId
data = pd.merge(employees, roles, how='left', on='RoleId')
# data = data.drop_duplicates('RoleName', keep='first')

# Handle missing termination dates
today = datetime.today().strftime('%Y-%m-%d')
data['TerminationDate'].fillna(today, inplace=True)

# Calculate tenure
data['EmploymentDate'] = pd.to_datetime(data['EmploymentDate'])
data['TerminationDate'] = pd.to_datetime(data['TerminationDate'])
data['Tenure'] = (data['TerminationDate'] - data['EmploymentDate']).dt.days

data.head()

,EmploymentDate,TerminationDate,Salary,RoleId,RoleName,Tenure
0,2022-06-15,2023-05-30,39991.40,166576,"Engineer, communications",349
1,2021-07-13,2023-05-30,99426.36,610063,Horticultural consultant,686
2,2018-07-05,2019-10-27,85841.31,421751,"Surveyor, hydrographic",479
3,2018-07-29,2023-05-30,81520.31,907181,Estate manager/land agent,1766
4,2016-12-23,2023-05-30,84760.28,725699,Probation officer,2349


In [12]:
# Cell 4: Split the data
# Convert categorical data to numeric
data['RoleName'] = data['RoleName'].astype('category').cat.codes

X = data[['RoleName', 'Tenure']]
y = data['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Load data
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Define the model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 64),
    nn.ReLU(),
    nn.Dropout(0.5),  # Add dropout layer
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Cell 5: Train the model
n_epochs = 10
for epoch in range(n_epochs):
    for inputs, targets in train_loader:
        # Forward pass
        output = model(inputs)
        loss = criterion(output, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss for every epoch
    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 866880640.0
Epoch: 2, Loss: 699918208.0
Epoch: 3, Loss: 809349696.0
Epoch: 4, Loss: 715106304.0
Epoch: 5, Loss: 763719424.0
Epoch: 6, Loss: 837620736.0
Epoch: 7, Loss: 609408256.0
Epoch: 8, Loss: 654520064.0
Epoch: 9, Loss: 754658176.0
Epoch: 10, Loss: 769740160.0


In [14]:
# Cell 6: Evaluate the model
model.eval()
with torch.no_grad():
    total_loss = 0
    total_count = 0
    for inputs, targets in test_loader:
        output = model(inputs)
        loss = criterion(output, targets)
        total_loss += loss.item() * inputs.size(0)
        total_count += inputs.size(0)
    mse = total_loss / total_count
    print('MSE:', mse)

MSE: 675335654.32832


In [15]:
import numpy as np
rmse = np.sqrt(mse)
print('RMSE:', rmse)

RMSE: 25987.220981249997


In [16]:
# Cell 7: Save the model
if not os.path.exists('models'):
    os.makedirs('models')

dummy_input = torch.randn(1, X_train.shape[1])
torch.onnx.export(model, dummy_input, "models/salary_predictor.onnx")

data.to_csv('models/data.csv', sep=',')

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [20]:
# Assuming `new_data` is your new input data as a DataFrame
new_data = pd.DataFrame({
    'RoleName': ['Assistant', 'Analyst'],
    'Tenure': [3, 10000],
})

# print the samples
print(new_data)

# Remember to preprocess the new data the same way as the training data
new_data['RoleName'] = new_data['RoleName'].astype('category').cat.codes
new_data = scaler.transform(new_data)  # Using the same scaler used for training
new_data = torch.tensor(new_data, dtype=torch.float32)

# Now we can predict with the model
model.eval()
with torch.no_grad():
    predictions = model(new_data)

# Convert the predictions to a numpy array and print them
predictions_np = predictions.numpy()

for i, prediction in enumerate(predictions_np):
    print(f"Prediction for sample {i}: {prediction[0]}")

    RoleName  Tenure
0  Assistant       3
1    Analyst   10000
Prediction for sample 0: 74019.7578125
Prediction for sample 1: 73216.8671875
